In [39]:
import os
os.environ['OPENAI_API_KEY'] = "EXAMPLE"

# 도구
##### 도구는 에이전트가 외부 인터페이스와 상호작용하는데 사용할 수 있는 함수이다.
##### 해당 도구들은 일반적인 유틸리티(검색, 연산 등), 다른 체인, 또는 다른 에이전트도 될 수 있다.
##### 특화된 기능을 처리하기 위한 도구

## 기본 도구
##### 내장 도구 예시

In [16]:
!pip install -q langchain_community
!pip install -q wikipedia
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 13.6 MB/s eta 0:00:00


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

##### 도구 초기화

In [6]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [7]:
print(tool.name)
print(tool.description)
print(tool.args)
print(tool.return_direct)
print(tool.run({"query": "langchain"}))
print(tool.run("langchain"))

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
{'query': {'title': 'Query', 'type': 'string'}}
False
Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications 
Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications 


##### 도구에 따라 단일 입력/다중 입력 지원 여부가 다르다. (확인 필요)

##### 인수의 기본 제공 이름, 설명 및 JSON 스키마를 수정할 수 있다.

In [9]:
from langchain_core.pydantic_v1 import BaseModel, Field


class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia, should be 3 or less words"
    )

In [10]:
tool = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

In [11]:
print(tool.name)
print(tool.description)
print(tool.args)
print(tool.return_direct)
print(tool.run("langchain"))

wiki-tool
look up things in wikipedia
{'query': {'title': 'Query', 'description': 'query to look up in Wikipedia, should be 3 or less words', 'type': 'string'}}
True
Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications 


## 툴킷
##### 특정 작업을 위해 함께 사용되도록 설계된 도구 모음
##### get_tools 를 사용하여 도구 목록을 반환하는 메서드를 제공한다.

```python
# Initialize a toolkit
toolkit = ExampleTookit(...)

# Get list of tools
tools = toolkit.get_tools()

# Create agent
agent = create_agent_method(llm, tools, prompt)
```

## 사용자 정의 도구
##### Langchain은 자신만의 에이전트를 구성할 때 사용할 수 있는 도구 목록을 제공한다.



1. @도구 데코레이터 : @tool 데코레이터를 사용하여 사용자 정의 도구를 정의
2. BaseTool 서브클래스 : BaseTool 클래스는 서브클래싱하여 사용자 정의 도구를 명시적으로 정의(도구 제어 최대화, 더 많은 작업 필요)
3. StructuredTool 데이터 클래스 : 앞선 두 방법을 혼합한 것. BaseTool 클래스에서 상속하는 것보다 더 편리하지만 데코레이터를 사용하는 것보다 더 많은 기능 제공



In [17]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

### @도구 데코레이터


In [18]:
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

In [19]:
print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}


In [21]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [22]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(a: int, b: int) -> int - Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


##### 도구 이름과 JSON 인수를 도구 데코레이터에 전달하여 사용자 정의할 수 있다.

In [20]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

### BaseTool 서브클래스

In [23]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"
    args_schema: Type[BaseModel] = SearchInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return "LangChain"

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")


class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = CalculatorInput
    return_direct: bool = True

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return a * b

    async def _arun(
        self,
        a: int,
        b: int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("Calculator does not support async")

In [24]:
search = CustomSearchTool()
print(search.name)
print(search.description)
print(search.args)

custom_search
useful for when you need to answer questions about current events
{'query': {'title': 'Query', 'description': 'should be a search query', 'type': 'string'}}


In [25]:
multiply = CustomCalculatorTool()
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

Calculator
useful for when you need to answer questions about math
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}
True


### StructuredTool 데이터 클래스


In [26]:
def search_function(query: str):
    return "LangChain"


search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events",
    # coroutine= ... <- you can specify an async method if desired as well
)

In [27]:
print(search.name)
print(search.description)
print(search.args)

Search
Search(query: str) - useful for when you need to answer questions about current events
{'query': {'title': 'Query', 'type': 'string'}}


##### args_schema 입력에 대한 추가 정보를 제공하기 위해 사용자 정의를 정의할 수 있다.

In [28]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
    # coroutine= ... <- you can specify an async method if desired as well
)

In [29]:
print(calculator.name)
print(calculator.description)
print(calculator.args)

Calculator
Calculator(a: int, b: int) -> int - multiply numbers
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}


### 도구 오류
##### 도구에 오류가 발생했을 때, 예외가 설정되어 있지 않으면 에이전트는 실행을 중지한다.
##### 그래서, 에이전트가 계속 실행되도록 하려면 ToolException을 발생시키고 그에 따라 handle_tool_error를 설정할 수 있다.



1. handle_tool_error : True 로 설정
2. 통합된 문자열 값으로 설정
3. 사용자 정의 함수로 설정 : 해당 함수는 ToolException 매개변수로 사용하여 str 값을 반환해야 함.



In [33]:
from langchain_core.tools import ToolException


def search_tool1(s: str):
    raise ToolException("The search tool1 is not available.")

##### handle_tool_error를 설정하지 않을 때 오류가 발생함.

In [35]:
search = StructuredTool.from_function(
    func=search_tool1,
    name="search_tool1",
    description="A bad tool",
)

search.run("test")

ToolException: The search tool1 is not available.

In [36]:
search = StructuredTool.from_function(
    func=search_tool1,
    name="search_tool1",
    description="A bad tool",
    handle_tool_error=True,
)

search.run("test")

'The search tool1 is not available.'

In [37]:
def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool."
    )


search = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="A bad tool",
    handle_tool_error=_handle_error,
)

search.run("test")

'The following errors occurred during tool execution:The search tool1 is not available.Please try another tool.'

## 도구를 OpenAI 기능으로 사용하기

In [38]:
!pip install -qU langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [40]:
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI

In [41]:
model = ChatOpenAI()

In [42]:
tools = [MoveFileTool()]
functions = [convert_to_openai_function(t) for t in tools]

In [43]:
functions[0]

{'name': 'move_file',
 'description': 'Move or rename a file from one location to another',
 'parameters': {'type': 'object',
  'properties': {'source_path': {'description': 'Path of the file to move',
    'type': 'string'},
   'destination_path': {'description': 'New path for the moved file',
    'type': 'string'}},
  'required': ['source_path', 'destination_path']}}

In [44]:
message = model.invoke(
    [HumanMessage(content="move file foo to bar")], functions=functions
)

In [45]:
message

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"source_path":"foo","destination_path":"bar"}', 'name': 'move_file'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 77, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'function_call', 'logprobs': None}, id='run-2c613b79-8410-446a-91fd-cd38632d776a-0')

In [46]:
message.additional_kwargs["function_call"]

{'arguments': '{"source_path":"foo","destination_path":"bar"}',
 'name': 'move_file'}

##### bind_functions를 사용하면 OpenAI 채팅 모델에 함수형 객체를 자동으로 바인딩하고 변환한다.

In [47]:
model_with_functions = model.bind_functions(tools)
model_with_functions.invoke([HumanMessage(content="move file foo to bar")])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"source_path":"foo","destination_path":"bar"}', 'name': 'move_file'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 77, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'function_call', 'logprobs': None}, id='run-bbd98712-f09a-4c35-a0f2-3d802d2a89f0-0')

##### 또는 bind_tools를 사용하여 functions / function_call 대신에 tools / tool_choice를 사용하여 업데이트된 OpenAI를 사용할 수 있다.

In [48]:
model_with_tools = model.bind_tools(tools)
model_with_tools.invoke([HumanMessage(content="move file foo to bar")])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AbHejBvPyl0NlBvnMLKvwls8', 'function': {'arguments': '{"source_path":"foo","destination_path":"bar"}', 'name': 'move_file'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 77, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c4fb47da-a46f-4135-bf15-f30d48adc90e-0', tool_calls=[{'name': 'move_file', 'args': {'source_path': 'foo', 'destination_path': 'bar'}, 'id': 'call_AbHejBvPyl0NlBvnMLKvwls8'}])